<a href="https://colab.research.google.com/github/CodingDuckmx/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/Jesus_CaballeroMedrano_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [6]:
### YOUR CODE STARTS HERE

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-20 21:29:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2020-01-20 21:29:24 (127 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
#Fisrt we load the libraries we'll need.

import pandas as pd

from scipy.stats import ttest_1samp, ttest_ind


In [14]:
df = pd.read_csv('house-votes-84.data', header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
#We got no headers and the null values are ?

In [29]:
#Set the headers for our dataframe and change the nul values to NaN's

column_headers = ['party', 'handicapped-infants', 'water-project', 
'budget', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 
'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration', 
'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 
'south-africa']

df = pd.read_csv('house-votes-84.data', header=None, names=column_headers, na_values='?')

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [34]:
#Replace y (yes) for 1 and n (no)for 0.

df = df.replace({'y':1,'n':0})

df.head()



,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [63]:
# Split df into two by party

rep = df[df['party'] == 'republican']

dem = df[df['party']== 'democrat']


print(rep.head())

print(dem.head())


         party  handicapped-infants  ...  duty-free  south-africa
0   republican                  0.0  ...        0.0           1.0
1   republican                  0.0  ...        0.0           NaN
7   republican                  0.0  ...        NaN           1.0
8   republican                  0.0  ...        0.0           1.0
10  republican                  0.0  ...        0.0           0.0

[5 rows x 17 columns]
      party  handicapped-infants  water-project  ...  crime  duty-free  south-africa
2  democrat                  NaN            1.0  ...    1.0        0.0           0.0
3  democrat                  0.0            1.0  ...    0.0        0.0           1.0
4  democrat                  1.0            1.0  ...    1.0        1.0           1.0
5  democrat                  0.0            1.0  ...    1.0        1.0           1.0
6  democrat                  0.0            1.0  ...    1.0        1.0           1.0

[5 rows x 17 columns]


##Handicapped-Infants Bill

### 1) Null hypothesis: 

There's no difference between the average voting (level of support) for the `handicapped-infants` bill between the democrats and republicans in the House of Representatives.

i.e. $\bar{x}_1 = \bar{x}_2$

### 2) Alternative Hypothesis

The suppot between the two parties will differ.

i.e. $\bar{x}_1 \neq \bar{x}_2$

###3) Confidence level: 99%

In [46]:
ttest_ind(rep['handicapped-infants'],dem['handicapped-infants'], nan_policy='omit')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

The p-value is lower than 0.01 then we reject the null Hypothesis.

In [72]:
print(rep['handicapped-infants'].mean())
print(dem['handicapped-infants'].mean())

0.18787878787878787
0.6046511627906976


###The `handicapped-infants` bill is more supported by democrats than republicans.

##Water-project

### 1) Null hypothesis: 

There's no difference between the average voting (level of support) for the `water-project` bill between the democrats and republicans in the House of Representatives.

i.e. $\bar{x}_1 = \bar{x}_2$

### 2) Alternative Hypothesis

The suppot between the two parties will differ.

i.e. $\bar{x}_1 \neq \bar{x}_2$

###3) Confidence level: 99%

In [64]:
ttest_ind(rep['water-project'],dem['water-project'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

The p-value is greater than 0.01, then we cannot reject the null Hypothesis.

The `water-proyect` bill is equally supported by both parties.

##Budget

### 1) Null hypothesis: 

There's no difference between the average voting (level of support) for the `budget` bill between the democrats and republicans in the House of Representatives.

i.e. $\bar{x}_1 = \bar{x}_2$

### 2) Alternative Hypothesis

The suppot between the two parties will differ.

i.e. $\bar{x}_1 \neq \bar{x}_2$

###3) Confidence level: 99%

In [67]:
ttest_ind(rep['budget'],dem['budget'],nan_policy='omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

The p-value is lower than 0.01 then we reject the null Hypothesis.

In [73]:
print(rep['budget'].mean())
print(dem['budget'].mean())

0.13414634146341464
0.8884615384615384


###The `budget` bill is more supported by democrats than republicans.

##Physician-fee-freeze

### 1) Null hypothesis: 

There's no difference between the average voting (level of support) for the `physician-free-freeze` bill between the democrats and republicans in the House of Representatives.

i.e. $\bar{x}_1 = \bar{x}_2$

### 2) Alternative Hypothesis

The suppot between the two parties will differ.

i.e. $\bar{x}_1 \neq \bar{x}_2$

###3) Confidence level: 99%

In [75]:
ttest_ind(rep['physician-fee-freeze'],dem['physician-fee-freeze'],nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

The p-value is lower than 0.01 then we reject the null Hypothesis.

In [76]:
print(rep['physician-fee-freeze'].mean())
print(dem['physician-fee-freeze'].mean())

0.9878787878787879
0.05405405405405406


###The `physician fee freeze` bill is more supported by republicans than democrats.

#Goals:

1) The `budget bill` is more supported by democrats than republicans with a $p<0.01$.

2) The `physician fee freeze bill` is more supported by republicans than democrats with a $p<0.01$.

3) The `water project bill` is equally supported by both parties with a $p>0.01$.